In [11]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
import scipy.stats as st
from scipy.stats import linregress

In [3]:
measles_raw_data = pd.read_csv("measles - measles.csv")
measles_raw_data.head()
measles_raw_data.groupby("year").count()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,index,state,name,type,city,county,district,enroll,mmr,overall,xrel,xmed,xper,lat,lng
year,,,,,,,,,,,,,,,
2017,1939,1939,1939,0,1939,0,0,1939,1939,1939,0,0,0,1939,1939
2017-18,10418,10418,10418,7183,6060,10390,0,4970,10418,10418,9,1829,2698,10410,10410
2018-19,48075,48075,48075,22308,34573,43788,0,40900,48075,48075,84,17723,4483,46539,46539


In [4]:
measles_clean = measles_raw_data.drop(columns=["xper", "xmed", "xrel", "district", "year", "county", "enroll"])
# measles_clean = measles_clean.dropna()

# measles_clean.dtypes

In [5]:
# neg_count = measles_clean.loc[["mmr" == "-1"]]
# len(neg_count)

measles_clean['mmr'].value_counts()
measles_clean['overall'].value_counts()

-1.000000      27224
 98.000000      5191
 100.000000     3619
 95.000000      3051
 99.000000      1764
               ...  
 90.140845         1
 96.020000         1
 43.640000         1
 98.098859         1
 90.052356         1
Name: overall, Length: 2691, dtype: int64

In [6]:
measles_clean = measles_clean.loc[measles_clean["mmr"] >= 0]
measles_clean = measles_clean.loc[measles_clean["overall"] >= 0]
# measles_clean

In [7]:
measles_clean["type"].fillna("Other", inplace = True)
measles_clean

,index,state,name,type,city,mmr,overall,lat,lng
2018,1,California,Abby Reinke Elementary,Public,Temecula,99.00,96.00,33.492008,-117.087291
2019,2,California,Abraham Lincoln Elementary,Public,Santa Ana,99.00,99.00,33.741155,-117.902009
2020,2,California,Abraham Lincoln Elementary,Public,Santa Ana,99.00,99.00,34.077328,-117.317020
2021,2,California,Abraham Lincoln Elementary,Public,Santa Ana,99.00,99.00,34.227892,-118.252604
2022,2,California,Abraham Lincoln Elementary,Public,Santa Ana,99.00,99.00,33.734440,-116.373073
...,...,...,...,...,...,...,...,...,...
63242,1364,Washington,Overcomer Academy,Other,Auburn,23.53,17.65,47.301117,-122.276320
63243,1365,Washington,Salvation Christian Academy- Formerly Slavic C...,Other,Edgewood,22.22,22.22,47.249023,-122.287615
63244,1366,Washington,Saratoga,Other,Stanwood,20.00,20.00,48.242579,-122.359097
63245,1367,Washington,Skagit Family Learning Center,Other,Mount Vernon,17.65,11.76,48.406426,-122.339166


In [8]:
measles_clean.groupby("type").count()

,index,state,name,city,mmr,overall,lat,lng
type,,,,,,,,
Kindergarten,1484,1484,1484,0,1484,1484,1484,1484
Other,3122,3122,3122,1978,3122,3122,3121,3121
Private,2258,2258,2258,2237,2258,2258,2207,2207
Public,14904,14904,14904,14904,14904,14904,14895,14895


In [9]:
# Melissa's code starts here

In [55]:
# Set up Public, Private, Kindergarten, and Other dataframes

public_stats = measles_clean.loc[measles_clean["type"] == "Public"]
# public_stats

private_stats = measles_clean.loc[measles_clean["type"] == "Private"]
# private_stats

kindergarten_stats = measles_clean.loc[measles_clean["type"] == "Kindergarten"]
# kindergarten_stats

other_stats = measles_clean.loc[measles_clean["type"] == "Other"]
# other_stats

colorado = measles_clean.loc[measles_clean["state"] == "Colorado"]
colorado.groupby("type").count()

,index,state,name,city,mmr,overall,lat,lng
type,,,,,,,,
Kindergarten,1484,1484,1484,0,1484,1484,1484,1484
Private,21,21,21,0,21,21,21,21


In [60]:
# Loop through the data frame to calculate stats by state

# Set up empty lists to fill in for loop
states = measles_clean["state"].unique()
# states
count_states = []
mean_mmr = []
median_mmr = []
public_mean = []
public_median = []
private_mean = []
private_median = []
kind_mean = []
kind_median = []
other_mean = []
other_median = []

for state in states:
    
    # Identify what state to work with
    state_active = measles_clean.loc[measles_clean["state"] == state]
    
    # Count all the schools for each state
    count_active = state_active["state"].count()
    count_states.append(count_active)
    
    # Calculate mean for each state
    mean_active = state_active["mmr"].mean()
    mean_mmr.append(mean_active)
    
    # Calculate median for each state
    median_active = state_active["mmr"].median()
    median_mmr.append(median_active)
    
    # Reset state_active for use on public subsets
    state_active = public_stats.loc[public_stats["state"] == state]
    
    # Calculate mean for public schools
    public_active = state_active["mmr"].mean()
    public_mean.append(public_active)
    
    # Calculate median for public schools
    public_active = state_active["mmr"].median()
    public_median.append(public_active)
    
    # Reset state_active for use on private subsets
    state_active = private_stats.loc[private_stats["state"] == state]
    
    # Calculate mean for private schools
    private_active = state_active["mmr"].mean()
    private_mean.append(private_active)
    
    # Calculate median for private schools
    private_active = state_active["mmr"].median()
    private_median.append(private_active)
    
    # Reset state_active for use on kindergarten subsets
    state_active = kindergarten_stats.loc[kindergarten_stats["state"] == state]
    
    # Calculate mean for kindergarten schools
    kind_active = state_active["mmr"].mean()
    kind_mean.append(kind_active)
    
    # Calculate median for kindergarten schools
    kind_active = state_active["mmr"].median()
    kind_median.append(kind_active)
    
    # Reset state_active for use on kindergarten subsets
    state_active = kindergarten_stats.loc[kindergarten_stats["state"] == state]
    
    # Calculate mean for Other schools
    other_active = state_active["mmr"].mean()
    other_mean.append(other_active)
    
    # Calculate median for Other schools
    other_active = state_active["mmr"].median()
    other_median.append(other_active)
   
       
other_median

[98.0, nan, 94.74, nan, nan, nan]

In [61]:
# Get list of states to work with

# Set up summary table of averages per state
avg_by_state = pd.DataFrame({
    "State": states
})

avg_by_state["Count of Schools"] = count_states
avg_by_state["Overall  Mean"] = mean_mmr
avg_by_state["Overall Median"] = median_mmr
avg_by_state["Public Mean"] = public_mean
avg_by_state["Public Median"] = public_median
avg_by_state["Private Mean"] = private_mean
avg_by_state["Private Median"] = private_median
avg_by_state["Kindergarten Mean"] = kind_mean
avg_by_state["Kindergarten Median"] = kind_median
avg_by_state["Other Mean"] = other_mean
avg_by_state["Other Median"] = other_median

avg_by_state

# Investigate NaNs
# Indicate a lack of that school type reporting out of that state
# Hmm, no, that doesn't seem right because Colorado has kindergartens

,State,Count of Schools,Overall Mean,Overall Median,Public Mean,Public Median,Private Mean,Private Median,Kindergarten Mean,Kindergarten Median,Other Mean,Other Median
0,California,14224,96.371836,98.000000,96.644004,98.00,96.644004,98.00,96.644004,98.00,96.644004,98.00
1,Colorado,1505,94.210904,97.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ohio,2917,91.463692,94.870000,91.284485,94.74,91.284485,94.74,91.284485,94.74,91.284485,94.74
3,Oregon,806,94.081152,95.677233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vermont,338,94.583320,97.222222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Washington,1978,89.336830,92.125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
# This was research

# # measles_clean.loc[(measles_clean.type == "NaN"), "type"]="Other"
# # measles_clean["type"]=measles_clean["type"].replace({"NaN":"Other"})
# # measles_clean.loc[(measles_clean.type == 'NaN'),'type']='Other'
# # measles_clean.fillna(0)

# measles_clean["type"]=measles_clean["type"].fillna(0)
# measles_clean["type"] = measles_clean.type.astype(str)

# measles_clean.loc[(measles_clean.type == "0"), "type"]="Other"
# measles_clean